# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [48]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [49]:
city_csv = "city_df.csv"
city_df = pd.read_csv("city_df.csv")

city_df.head()




,Unnamed: 0,CityID,City,Humidity,Temp,Latitude,Cloudiness,WindSpeed,MaxTemp,Longitude,LatLong,CountryCode
0,0,2127202,Anadyr,75,-16.31,64.75,66,6.71,-16.01,177.48,"{'lon': 177.48, 'lat': 64.75}",RU
1,1,2119283,Zyryanka,97,-25.53,65.75,97,5.26,-25.53,150.85,"{'lon': 150.85, 'lat': 65.75}",RU
2,2,3430863,Mar del Plata,47,79.25,-38.00,0,9.17,80.01,-57.56,"{'lon': -57.56, 'lat': -38}",AR
3,3,2206939,Bluff,92,60.01,-46.60,100,5.99,60.01,168.33,"{'lon': 168.33, 'lat': -46.6}",NZ
4,4,3576994,Cockburn Town,74,76.05,21.46,0,18.05,76.05,-71.14,"{'lon': -71.14, 'lat': 21.46}",TC


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [50]:
locations = city_df[["Latitude","Longitude"]]
humidity = city_df["Humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=10, 
                                 point_radius=1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [57]:
maxtemp_df = city_df[(city_df['MaxTemp'] < 80) & (city_df['MaxTemp'] > 70)]


windspeed_df = maxtemp_df[(maxtemp_df['WindSpeed'] < 20)]
perfect_df = windspeed_df[(windspeed_df['Cloudiness'] == 0)].dropna()
perfect_df

,Unnamed: 0,CityID,City,Humidity,Temp,Latitude,Cloudiness,WindSpeed,MaxTemp,Longitude,LatLong,CountryCode
4,4,3576994,Cockburn Town,74,76.05,21.46,0,18.05,76.05,-71.14,"{'lon': -71.14, 'lat': 21.46}",TC
26,26,3374210,Sao Filipe,78,70.38,14.90,0,8.77,70.38,-24.50,"{'lon': -24.5, 'lat': 14.9}",CV
49,49,2075265,Busselton,32,75.00,-33.65,0,7.34,79.00,115.33,"{'lon': 115.33, 'lat': -33.65}",AU
89,89,3860443,Comodoro Rivadavia,45,70.43,-45.87,0,12.50,70.43,-67.50,"{'lon': -67.5, 'lat': -45.87}",AR
280,280,2137773,Vao,65,76.28,-22.67,0,13.35,76.28,167.48,"{'lon': 167.48, 'lat': -22.67}",NC
448,448,1255619,Srivardhan,65,71.29,18.03,0,3.85,71.29,73.02,"{'lon': 73.02, 'lat': 18.03}",IN


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [66]:
hotel_df = pd.DataFrame(columns= ['hotel_name','CityID']) 


for index,row in perfect_df.iterrows():
    target_coordinates = f"{row['Latitude']},{row['Longitude']}"
    target_search = "Hotels"
    target_radius = 5000
    target_type = "lodging"
    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key}

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)

    # convert response to json
    response = response.json()
    
    hotel_df = hotel_df.append({'hotel_name' : response["results"][0]["name"],
                                'CityID' : row['CityID']}, ignore_index=True)
hotel_df

,hotel_name,CityID
0,Bohio Dive Resort,3576994
1,Hotel Xaguate,3374210
2,Abbey Beach Resort,2075265
3,WAM Hotel Patagónico,3860443
4,OURE TERA beach resort,2137773
5,Tranquil Beach Resort,1255619


In [76]:
BestHotels = pd.merge(hotel_df, perfect_df, on="CityID")

BestHotels.set_index("CityID", inplace=True)
BestHotels

,hotel_name,Unnamed: 0,City,Humidity,Temp,Latitude,Cloudiness,WindSpeed,MaxTemp,Longitude,LatLong,CountryCode
CityID,,,,,,,,,,,,
3576994,Bohio Dive Resort,4,Cockburn Town,74,76.05,21.46,0,18.05,76.05,-71.14,"{'lon': -71.14, 'lat': 21.46}",TC
3374210,Hotel Xaguate,26,Sao Filipe,78,70.38,14.90,0,8.77,70.38,-24.50,"{'lon': -24.5, 'lat': 14.9}",CV
2075265,Abbey Beach Resort,49,Busselton,32,75.00,-33.65,0,7.34,79.00,115.33,"{'lon': 115.33, 'lat': -33.65}",AU
3860443,WAM Hotel Patagónico,89,Comodoro Rivadavia,45,70.43,-45.87,0,12.50,70.43,-67.50,"{'lon': -67.5, 'lat': -45.87}",AR
2137773,OURE TERA beach resort,280,Vao,65,76.28,-22.67,0,13.35,76.28,167.48,"{'lon': 167.48, 'lat': -22.67}",NC
1255619,Tranquil Beach Resort,448,Srivardhan,65,71.29,18.03,0,3.85,71.29,73.02,"{'lon': 73.02, 'lat': 18.03}",IN


In [77]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel_name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{CountryCode}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in BestHotels.iterrows()]
locations = BestHotels[["Latitude", "Longitude"]]


In [86]:
# Add marker layer ontop of heat map


# Display Map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)


fig = gmaps.figure()
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))